In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import to_date, lit, date_format

try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .appName("Pos-Cash") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "2") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/21 18:04:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
pos = spark.read.csv("/data/raw/POS_CASH_balance.csv",
                               header=True,
                               inferSchema=True)

pos.createOrReplaceTempView("pos")

# Contagem de linhas e colunas
num_rows = pos.count()
num_columns = len(pos.columns)

print(f'Quantidade de linhas: {num_rows}')
print(f'Quantidade de variaveis (colunas): {num_columns}')

pos.show(5, truncate=False)

Quantidade de linhas: 10001358
Quantidade de variaveis (colunas): 8
+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|CNT_INSTALMENT|CNT_INSTALMENT_FUTURE|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|1803195   |182943    |-31           |48.0          |45.0                 |Active              |0     |0         |
|1715348   |367990    |-33           |36.0          |35.0                 |Active              |0     |0         |
|1784872   |397406    |-32           |12.0          |9.0                  |Active              |0     |0         |
|1903291   |269225    |-35           |48.0          |42.0                 |Active              |0     |0         |
|2341044   |334279    |-35           |36.0          |35.0                 |Active              |0     |0         |
+----------+

## Função para renomear e padronizar palavras

In [3]:
#Substituir espaço por underline e colocar a palavra em letras maiúsculas.
import re

def tratamento_palavra(palavra):
    palavra_transformada = re.sub(r'\s', '_', palavra).upper()

    return palavra_transformada  



## Renomeando e padronizando palavras da coluna NAME_CONTRACT_STATUS

In [4]:
# Extraindo os valores únicos diretamente do DataFrame
status = pos.select('NAME_CONTRACT_STATUS').distinct().collect()

# Aplicando a função de tratamento de palavra a cada valor único
nova_lista = [tratamento_palavra(row['NAME_CONTRACT_STATUS']) for row in status]

# Exibindo a lista resultante
print(nova_lista)

[Stage 6:=======>                                                   (1 + 7) / 8]

['DEMAND', 'APPROVED', 'COMPLETED', 'RETURNED_TO_THE_STORE', 'ACTIVE', 'SIGNED', 'CANCELED', 'XNA', 'AMORTIZED_DEBT']


## Criando variáveis de janela temporal

In [5]:
## Habilitando uso do SparkSQL
pos.createOrReplaceTempView("dados")

df_temp_01 = spark.sql("""
SELECT
    *,
      CASE
        WHEN MONTHS_BALANCE >= -3 THEN 1
        ELSE 0
    END AS U3M,
    CASE
        WHEN MONTHS_BALANCE >= -6 THEN 1
        ELSE 0
    END AS U6M,  
    CASE
        WHEN MONTHS_BALANCE >= -12 THEN 1
        ELSE 0
    END AS U12M
FROM dados
ORDER BY `SK_ID_PREV`;
""")
df_temp_01.createOrReplaceTempView("df_temp_01")
display(df_temp_01.limit(5))

DataFrame[SK_ID_PREV: int, SK_ID_CURR: int, MONTHS_BALANCE: int, CNT_INSTALMENT: double, CNT_INSTALMENT_FUTURE: double, NAME_CONTRACT_STATUS: string, SK_DPD: int, SK_DPD_DEF: int, U3M: int, U6M: int, U12M: int]

## Criando variáveis de primeira camada

In [6]:
from pyspark.sql.functions import col, round, sum, avg, max, min, when, countDistinct, count, date_format, current_date
# Definindo as colunas para a agregação.
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('MONTHS_BALANCE')
colunas_agregacao_total.remove('NAME_CONTRACT_STATUS')

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M', 'U6M','U12M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for coluna in colunas_agregacao_total:
  # Verifica se a coluna atual não é uma coluna de flag.
  if not any(flag in coluna for flag in colunas_flags):
    for flag in colunas_flags:
      if 'DPD' in coluna:
        expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_TOT_{coluna.upper()}_{flag.upper()}_POS_CASH'))
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MED_{coluna.upper()}_{flag.upper()}_POS_CASH'))

      else:
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MED_{coluna.upper()}_{flag.upper()}_POS_CASH'))


# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_02 = df_temp_01.groupBy('SK_ID_PREV').agg(*expressoes_agregacao).orderBy('SK_ID_PREV')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_02.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_02.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_02.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_02.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostra as 5 primeiras linhas do DataFrame usando display
display(df_temp_02.limit(5))

Quantidade Total de Variáveis Criadas: 18
Nomes das Variáveis Criadas: ['VL_MED_CNT_INSTALMENT_U3M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U6M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U12M_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U3M_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U6M_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U12M_POS_CASH', 'QT_TOT_SK_DPD_U3M_POS_CASH', 'QT_MED_SK_DPD_U3M_POS_CASH', 'QT_TOT_SK_DPD_U6M_POS_CASH', 'QT_MED_SK_DPD_U6M_POS_CASH', 'QT_TOT_SK_DPD_U12M_POS_CASH', 'QT_MED_SK_DPD_U12M_POS_CASH', 'QT_TOT_SK_DPD_DEF_U3M_POS_CASH', 'QT_MED_SK_DPD_DEF_U3M_POS_CASH', 'QT_TOT_SK_DPD_DEF_U6M_POS_CASH', 'QT_MED_SK_DPD_DEF_U6M_POS_CASH', 'QT_TOT_SK_DPD_DEF_U12M_POS_CASH', 'QT_MED_SK_DPD_DEF_U12M_POS_CASH']




[Stage 11:>                                                         (0 + 8) / 8]

Quantidade de linhas do DataFrame: 936325
Quantidade de colunas do DataFrame: 19




DataFrame[SK_ID_PREV: int, VL_MED_CNT_INSTALMENT_U3M_POS_CASH: double, VL_MED_CNT_INSTALMENT_U6M_POS_CASH: double, VL_MED_CNT_INSTALMENT_U12M_POS_CASH: double, VL_MED_CNT_INSTALMENT_FUTURE_U3M_POS_CASH: double, VL_MED_CNT_INSTALMENT_FUTURE_U6M_POS_CASH: double, VL_MED_CNT_INSTALMENT_FUTURE_U12M_POS_CASH: double, QT_TOT_SK_DPD_U3M_POS_CASH: bigint, QT_MED_SK_DPD_U3M_POS_CASH: double, QT_TOT_SK_DPD_U6M_POS_CASH: bigint, QT_MED_SK_DPD_U6M_POS_CASH: double, QT_TOT_SK_DPD_U12M_POS_CASH: bigint, QT_MED_SK_DPD_U12M_POS_CASH: double, QT_TOT_SK_DPD_DEF_U3M_POS_CASH: bigint, QT_MED_SK_DPD_DEF_U3M_POS_CASH: double, QT_TOT_SK_DPD_DEF_U6M_POS_CASH: bigint, QT_MED_SK_DPD_DEF_U6M_POS_CASH: double, QT_TOT_SK_DPD_DEF_U12M_POS_CASH: bigint, QT_MED_SK_DPD_DEF_U12M_POS_CASH: double]

## Criando variáveis com base na coluna NAME_CONTRACT_STATUS

In [7]:
# Definindo as colunas para agregação.
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('MONTHS_BALANCE')
colunas_agregacao_total.remove('NAME_CONTRACT_STATUS')

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M', 'U6M','U12M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for categoria in nova_lista:
  for coluna in colunas_agregacao_total:
    # Verifica se a coluna atual não é uma coluna de flag.
    if not any(flag in coluna for flag in colunas_flags):
      for flag in colunas_flags:
        if 'DPD' in coluna:
          expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_TOT_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH'))
          expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MED_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH'))
        else:
          expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MED_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH'))  

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_03 = df_temp_01.groupBy('SK_ID_PREV').agg(*expressoes_agregacao).orderBy('SK_ID_PREV')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_03.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_03.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_03.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_03.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')


# Mostrando o novo DataFrame com as variáveis criadas.
display(df_temp_03.limit(5))

Quantidade Total de Variáveis Criadas: 162
Nomes das Variáveis Criadas: ['VL_MED_CNT_INSTALMENT_U3M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_U6M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_U12M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U3M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U6M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U12M_DEMAND_POS_CASH', 'QT_TOT_SK_DPD_U3M_DEMAND_POS_CASH', 'QT_MED_SK_DPD_U3M_DEMAND_POS_CASH', 'QT_TOT_SK_DPD_U6M_DEMAND_POS_CASH', 'QT_MED_SK_DPD_U6M_DEMAND_POS_CASH', 'QT_TOT_SK_DPD_U12M_DEMAND_POS_CASH', 'QT_MED_SK_DPD_U12M_DEMAND_POS_CASH', 'QT_TOT_SK_DPD_DEF_U3M_DEMAND_POS_CASH', 'QT_MED_SK_DPD_DEF_U3M_DEMAND_POS_CASH', 'QT_TOT_SK_DPD_DEF_U6M_DEMAND_POS_CASH', 'QT_MED_SK_DPD_DEF_U6M_DEMAND_POS_CASH', 'QT_TOT_SK_DPD_DEF_U12M_DEMAND_POS_CASH', 'QT_MED_SK_DPD_DEF_U12M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_U3M_APPROVED_POS_CASH', 'VL_MED_CNT_INSTALMENT_U6M_APPROVED_POS_CASH', 'VL_MED_CNT_INSTALMENT_U12M_APPROVED_POS_CASH', 'VL_MED_CNT_INSTALME

[Stage 15:===========================================>              (6 + 2) / 8]

Quantidade de linhas do DataFrame: 936325
Quantidade de colunas do DataFrame: 163




DataFrame[SK_ID_PREV: int, VL_MED_CNT_INSTALMENT_U3M_DEMAND_POS_CASH: double, VL_MED_CNT_INSTALMENT_U6M_DEMAND_POS_CASH: double, VL_MED_CNT_INSTALMENT_U12M_DEMAND_POS_CASH: double, VL_MED_CNT_INSTALMENT_FUTURE_U3M_DEMAND_POS_CASH: double, VL_MED_CNT_INSTALMENT_FUTURE_U6M_DEMAND_POS_CASH: double, VL_MED_CNT_INSTALMENT_FUTURE_U12M_DEMAND_POS_CASH: double, QT_TOT_SK_DPD_U3M_DEMAND_POS_CASH: bigint, QT_MED_SK_DPD_U3M_DEMAND_POS_CASH: double, QT_TOT_SK_DPD_U6M_DEMAND_POS_CASH: bigint, QT_MED_SK_DPD_U6M_DEMAND_POS_CASH: double, QT_TOT_SK_DPD_U12M_DEMAND_POS_CASH: bigint, QT_MED_SK_DPD_U12M_DEMAND_POS_CASH: double, QT_TOT_SK_DPD_DEF_U3M_DEMAND_POS_CASH: bigint, QT_MED_SK_DPD_DEF_U3M_DEMAND_POS_CASH: double, QT_TOT_SK_DPD_DEF_U6M_DEMAND_POS_CASH: bigint, QT_MED_SK_DPD_DEF_U6M_DEMAND_POS_CASH: double, QT_TOT_SK_DPD_DEF_U12M_DEMAND_POS_CASH: bigint, QT_MED_SK_DPD_DEF_U12M_DEMAND_POS_CASH: double, VL_MED_CNT_INSTALMENT_U3M_APPROVED_POS_CASH: double, VL_MED_CNT_INSTALMENT_U6M_APPROVED_POS_CASH: do

## Realizando o join das tabelas criadas

In [8]:
from pyspark.sql.functions import col, round, sum, avg, max, min, when, countDistinct, count, date_format, current_date
# Fazendo o join das duas tabelas criadas.
df_temp_04 = df_temp_02.join(df_temp_03, 'SK_ID_PREV')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_04.columns
nomes_cols_novas = nomes_cols[1:-2]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_04.columns) - 3)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_04.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_04.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
display(df_temp_04.limit(5))

Quantidade Total de Variáveis Criadas: 178
Nomes das Variáveis Criadas: ['VL_MED_CNT_INSTALMENT_U3M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U6M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U12M_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U3M_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U6M_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U12M_POS_CASH', 'QT_TOT_SK_DPD_U3M_POS_CASH', 'QT_MED_SK_DPD_U3M_POS_CASH', 'QT_TOT_SK_DPD_U6M_POS_CASH', 'QT_MED_SK_DPD_U6M_POS_CASH', 'QT_TOT_SK_DPD_U12M_POS_CASH', 'QT_MED_SK_DPD_U12M_POS_CASH', 'QT_TOT_SK_DPD_DEF_U3M_POS_CASH', 'QT_MED_SK_DPD_DEF_U3M_POS_CASH', 'QT_TOT_SK_DPD_DEF_U6M_POS_CASH', 'QT_MED_SK_DPD_DEF_U6M_POS_CASH', 'QT_TOT_SK_DPD_DEF_U12M_POS_CASH', 'QT_MED_SK_DPD_DEF_U12M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U3M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_U6M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_U12M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U3M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U6M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U12M_DEMAND_POS_CA

[Stage 23:===========================================>              (6 + 2) / 8]

Quantidade de linhas do DataFrame: 936325
Quantidade de colunas do DataFrame: 181




DataFrame[SK_ID_PREV: int, VL_MED_CNT_INSTALMENT_U3M_POS_CASH: double, VL_MED_CNT_INSTALMENT_U6M_POS_CASH: double, VL_MED_CNT_INSTALMENT_U12M_POS_CASH: double, VL_MED_CNT_INSTALMENT_FUTURE_U3M_POS_CASH: double, VL_MED_CNT_INSTALMENT_FUTURE_U6M_POS_CASH: double, VL_MED_CNT_INSTALMENT_FUTURE_U12M_POS_CASH: double, QT_TOT_SK_DPD_U3M_POS_CASH: bigint, QT_MED_SK_DPD_U3M_POS_CASH: double, QT_TOT_SK_DPD_U6M_POS_CASH: bigint, QT_MED_SK_DPD_U6M_POS_CASH: double, QT_TOT_SK_DPD_U12M_POS_CASH: bigint, QT_MED_SK_DPD_U12M_POS_CASH: double, QT_TOT_SK_DPD_DEF_U3M_POS_CASH: bigint, QT_MED_SK_DPD_DEF_U3M_POS_CASH: double, QT_TOT_SK_DPD_DEF_U6M_POS_CASH: bigint, QT_MED_SK_DPD_DEF_U6M_POS_CASH: double, QT_TOT_SK_DPD_DEF_U12M_POS_CASH: bigint, QT_MED_SK_DPD_DEF_U12M_POS_CASH: double, VL_MED_CNT_INSTALMENT_U3M_DEMAND_POS_CASH: double, VL_MED_CNT_INSTALMENT_U6M_DEMAND_POS_CASH: double, VL_MED_CNT_INSTALMENT_U12M_DEMAND_POS_CASH: double, VL_MED_CNT_INSTALMENT_FUTURE_U3M_DEMAND_POS_CASH: double, VL_MED_CNT_INS

In [9]:
(df_temp_04.repartition(1)
     .write
     .mode("overwrite")
     .option("compression", "snappy")
     .parquet('/data/books/pos-cash'))

25/07/21 18:05:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

In [10]:
spark.stop()